In [20]:
#!pip install keras-tuner

In [31]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras


from kerastuner.tuners  import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
(train_images,train_labels),(test_images,test_labels) = keras.datasets.fashion_mnist.load_data()

In [23]:
train_images.shape

(60000, 28, 28)

In [24]:
train_images = train_images / 255
test_images = test_images / 255

In [25]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [26]:
train_images.shape

(60000, 28, 28, 1)

In [33]:
def build_model(hp):
  model = keras.Sequential([
                keras.layers.Conv2D(
                    filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                    kernel_size = hp.Choice('conv_1_kernel',values=[3,5]),
                    activation='relu',
                    input_shape=(28,28,1)
                ),
                keras.layers.Conv2D(
                    filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                    kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                    activation='relu'
                 ),
               keras.layers.Flatten(),
               keras.layers.Dense(
                          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                          activation='relu'
               ),
               keras.layers.Dense(10, activation='softmax')

               
    ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [34]:
tuner = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                     project_name="Mnist Fashion")

In [35]:
tuner.search_space_summary()

In [37]:
tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.5159 - accuracy: 0.8307 - val_loss: 0.3583 - val_accuracy: 0.8657
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3592 - accuracy: 0.8668 - val_loss: 0.3428 - val_accuracy: 0.8707
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3240 - accuracy: 0.8806 - val_loss: 0.3718 - val_accuracy: 0.8618


Epoch 1/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.5199 - accuracy: 0.8160 - val_loss: 0.3888 - val_accuracy: 0.8607
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.3720 - accuracy: 0.8635 - val_loss: 0.3899 - val_accuracy: 0.8665
Epoch 3/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.3362 - accuracy: 0.8770 - val_loss: 0.3841 - val_accuracy: 0.8630


Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.5605 - accuracy: 0.7955 - val_loss: 0.4363 - val_accuracy: 0.8373
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.4259 - accuracy: 0.8412 - val_loss: 0.4320 - val_accuracy: 0.8412
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3827 - accuracy: 0.8579 - val_loss: 0.4070 - val_accuracy: 0.8563


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3893 - accuracy: 0.8583 - val_loss: 0.2809 - val_accuracy: 0.8968
Epoch 2/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2447 - accuracy: 0.9089 - val_loss: 0.2437 - val_accuracy: 0.9078
Epoch 3/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.1847 - accuracy: 0.9302 - val_loss: 0.2297 - val_accuracy: 0.9168


Epoch 1/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4183 - accuracy: 0.8493 - val_loss: 0.3107 - val_accuracy: 0.8847
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2711 - accuracy: 0.9006 - val_loss: 0.2631 - val_accuracy: 0.8980
Epoch 3/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2179 - accuracy: 0.9185 - val_loss: 0.2562 - val_accuracy: 0.9065


INFO:tensorflow:Oracle triggered exit


In [38]:
tuner.results_summary()

In [39]:
models_list = tuner.get_best_models(num_models=2)
models_list

In [42]:
model = models_list[0]
model.evaluate(test_images,test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.2525 - accuracy: 0.9112


[0.2524717450141907, 0.9111999869346619]

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,308,826
Trainable params: 2,308,826
Non-trainable params: 0
_________________________________________________________________


In [44]:
models_list[1].evaluate(test_images,test_labels)

313/313 [==============================] - 1s 5ms/step - loss: 0.2747 - accuracy: 0.9034


[0.2747110426425934, 0.9034000039100647]

In [47]:
model.get_config()

{'build_input_shape': TensorShape([None, 28, 28, 1]),
 'layers': [{'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 28, 28, 1),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'dtype': 'float32',
    'filters': 64,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'conv2d',
    'padding': 'valid',
    'strides': (1, 1),
    'trainable': True,
    'use_bias': True}},
  {'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_